# CNN Text Classification Lab

## Goal
본 실습의 목표는 Convolutional Neural Network을 이용하여 문장을 여러 카테고리 중 하나로 분류하는 모델을 만드는 것입니다. 또한, 미리 학습된 단어 벡터를 모델에 적용하는 방법도 배워볼 것입니다.

## Dataset

학습 데이터는 Stanford 대학에서 구성한 공손함 데이터를 사용하겠습니다.

In [0]:
import os
import pandas as pd
import numpy as np
from collections import Counter
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.python.keras.preprocessing import sequence
from tensorflow import keras

nltk.download('punkt') ## 단어 토큰화

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
if not os.path.exists("Stanford_politeness_corpus.zip"):
  !wget http://www.cs.cornell.edu/~cristian/Politeness_files/Stanford_politeness_corpus.zip

if not os.path.exists("Stanford_politeness_corpus/wikipedia.annotated.csv"):
  !unzip Stanford_politeness_corpus.zip

In [0]:
def load_data(data_file): ## 데이터 불러옴 아까 했던거 상위25 하위25 자른거
  data = pd.read_csv(data_file)

  # Only use the top quartile as polite, and bottom quartile as impolite. Discard the rest.
  quantiles = data["Normalized Score"].quantile([0.25, 0.5, 0.75])
  print(quantiles)

  for i in range(len(data)):
    score = data.loc[i, "Normalized Score"]
    if score <= quantiles[0.25]:
      # Bottom quartile (impolite).
      data.loc[i, "Normalized Score"] = 0
    elif score >= quantiles[0.75]:
      # Top quartile (polite).
      data.loc[i, "Normalized Score"] = 1
    else:
      # Neutral.
      data.loc[i, "Normalized Score"] = 2

  data["Normalized Score"] = data["Normalized Score"].astype(int)

  # Discard neutral examples.
  data = data[data["Normalized Score"] < 2]
  data = data.sample(frac=1).reset_index(drop=True)

  return data

In [0]:
data = load_data("Stanford_politeness_corpus/wikipedia.annotated.csv")
pd.set_option('display.max_columns', None)

print(data.head())

0.25   -0.442633
0.50    0.052339
0.75    0.514399
Name: Normalized Score, dtype: float64
   Community      Id                                            Request  \
0  Wikipedia  620599  Another admin has already deleted the article....   
1  Wikipedia  615680  "Odder" in what sense? Also, why are you still...   
2  Wikipedia  132503  That's a couple levels of automation beyond my...   
3  Wikipedia  620837  CIV, I'll grant (and yes, I shouldn't have). B...   
4  Wikipedia  549682  As I wrote above, at first I thought lets keep...   

   Score1  Score2  Score3  Score4  Score5         TurkId1         TurkId2  \
0      22      19      21      14      18   ANGX5PAAYGL9P   AO5E3LWBYM72K   
1       9       9      15      13       9  A16PRU8T6NZLN5  A1BJTTNDDFZ3ZP   
2       9      13      13       9      17  A233ONYNWKDIYF  A2UFD1I8ZO1V4G   
3       9       7      13      17      12  A233ONYNWKDIYF  A21753FQKCM5DQ   
4      21      17      15      18      17  A1BQCRF5Q76YFY   AO3XB5I5QNNUI 

다음으로 할 일은 사전을 구성하는 것입니다.


신경망의 입력으로 사용하기 위해서는 문장을 숫자로 바꿔야 하는데, 사전의 역할은 단어를 숫자로, 숫자를 단어로 바꿔주는 것입니다.

여기서 빠른 계산을 위해 dictionary 자료 구조를 사용하는 것이 일반적입니다.



1.   문장들을 소문자로 바꾸고, tokenization (nltk.tokenize 패키지의 word_tokenize  함수 활용)
2.   전체 데이터에서 각 토큰들의 등장 빈도 확인 (collections 패키지의 Counter  클래스 활용)
3.   가장 등장 빈도가 높은 단어를 vocab_size 만큼 선택 (Counter의 most_common 함수 활용)
4.   각각의 단어에 고유한 숫자 부여. 이때, 0번째 토큰은 "<PAD>", 1번째 토큰은 "<OOV>" 할당
5.   토큰 -> 숫자 변환을 위한 dictionary (word_index 변수에 할당)와, 숫자 -> 토큰 변환을 위한 dictionary (word_inverted_index 변수에 할당) 생성
  
  



단어 수집 단어에 따라 id만들어주는 사전

In [0]:
from nltk.tokenize import word_tokenize
from nltk import Text

vocab_size = 5000
## 가장 등장빈도높은 5천개
# we assign the first indices in the vocabulary to special tokens that we use
# for padding, and for indicating unknown words
pad_id = 0
## 패딩토큰에 0번째 아이디
oov_id = 1
# 아웃오브id 첫번째
index_offset = 1
## 나머지 아이디는 뒤로밀림

def make_vocab(sentences):  ##!! 중요 문장을 입력으로 받아
  word_counter = Counter()
  
  # Your code here
  for n in range(0,len(sentences)):
    for i in word_tokenize(sentences[n]):
      word_counter.update("i")
      print("=====")
      
      print(word_counter)
    
  most_common = word_counter.most_common() ## 파이썬 컬렉션 객체 카운터 딕셔너리구조  키가 없으면 카운트 0 할당 // 등장빈도 셀때 좋음, 
                                            ## count 다 끝나면 고빈도 단어 보여줌.
  print("고빈도 단어:")
  for k, v in most_common[:10]:
    print(k, ": ", v)
    
  vocab = {
      '<PAD>': pad_id,
      '<OOV>': oov_id
  }
  vocab[pad_id] = oov_id
  
  
  ## enumerate 인덱스도 같이 포문돌아서 찍어줌 start = index+offset+1
  
  # Your code here
  
  return vocab
  
  
sentences = data["Request"].tolist()
word_index = make_vocab(sentences)  ## 딕셔너리 텍스트  id
word_inverted_index = {} ##딕셔너리/리스트 id 에서 원래 텍스트

# Your code here

print("\n단어 사전:")
for i in range(0, 10):
  print(i, word_inverted_index[i])
  
print("\n단어 사전 크기: ", len(word_index))







  
  ##문장이 들어오면 소문자 처리 문장을 토큰단위로 쪼개고 빈도 체크

In [0]:
from nltk.tokenize import word_tokenize
word_tokenize(data["Request"].tolist()[0])
# for i in word_tokenize(data["Request"].tolist()):
#   print(i)
  #word_counter[i] = 0

['See',
 '<',
 'url',
 '>',
 '.',
 'I',
 'assume',
 'you',
 'did',
 "n't",
 'mean',
 '<',
 'url',
 '>',
 '?']

사전이 잘 구성되었는지 시험해보겠습니다. 

사전이 잘 구성되고, 각각의 사전이 word_index 변수와 word_inverted_index 변수에 할당되었다면 문장이 숫자로 변환되었다가 다시 원래 문장으로 돌아오는 것을 확인하실 수 있습니다.

In [0]:
def index_to_text(indexes):
  return ' '.join([word_inverted_index[i] for i in indexes])
  
def text_to_index(text):
  tokens = tokens = word_tokenize(text.lower())
  indexes = []
  for tok in tokens:
    if tok in word_index:
      indexes.append(word_index[tok])
    else:
      indexes.append(oov_id)
      
  return indexes

print("원본: ", sentences[0])
ids = text_to_index(sentences[0])
print("문자 -> 숫자: ", ids)
print("숫자 -> 문자: ", index_to_text(ids))

다음으로, 숫자로 바뀐 문장들을 학습 데이터로 사용할 수 있도록 변형하겠습니다.



1.   모든 문장들을 동일한 길이가 되도록 padding 처리하거나 자름 (tensorflow.python.keras.preprocessing.sequence 패키지의 pad_sequence 함수 활용)
2.   데이터의 일부(10%)를 테스트 데이터로 분리



In [0]:
x_variable = # Your code here
## 모든 문장을 동일한 길이가 되도록 통일

sentence_size = 200
x_padded = sequence.pad_sequences(x_variable, ## x_variable 입력데이터, text - to index 배열안 배열
                                 maxlen=sentence_size, ## 200단위
                                 truncating='post', ##문자 자르거나 패딩할떄 어느부분남기고 자를지 
                                 padding='post',# post뒤 앞에서부터 토큰을 가지고있다가 뒷단 을 자르는거  패딩을 뒤에
                                 value=pad_id)


# 10%데이터를 시험문제로 따로 빼놓고 90퍼만 학습, 10퍼는 test데이터로
# Your code here

n_test = len(data)  // 10
test_inputs = x_padded[:n_test]
train_inputs = x_padded[n_test:]


ys = np.array(data["Normalized Score"].tolist())  # 정답데이터 불손0 공손1 리스트로 불러와서 케라스쓸라면 넘파이 써야되서 넘파이로 변형
## ys도 쪼개서 10% 테스트, => 넘파이 행렬 쉐입이나온다.

# Your code here

print("test_inputs shape: ", test_inputs.shape)
print("train_inputs shape: ", train_inputs.shape)
print("test_labels shape: ", test_labels.shape)
print("train_labels shape: ", train_labels.shape)

x_padded[0:10]  10: ㄲ끝까지

217개 문장이 200개 단어로

1961개 문장 200개단어


이제 모델을 설계할 차례입니다. 

keras.Sequential을 이용하여 CNN 모델을 구성해봅시다. Sequential 모델을 사용하려면 동일한 크기의 필터만 사용할 수 있습니다.

참고 함수: 

keras.layers.Embedding

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

keras.layers.Conv1D

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D

keras.layers.GlobalMaxPool1D

https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool1D

keras.layers.Dense

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense



In [0]:
## 1D 커널이 이동할때 1차원으로 세로만 움직인다 이느낌
## 글로벌맥스풀 커널 적용값 중 제이 큰값
##

# 임베딩 레이어 한국어는 형태소 단위로 임베딩 형태소끼리 결합해서 어절을 이룸 영어처럼 띄어쓰기 하면은 다양한 단어가 너무많음 같은집이여도 집은 집에 집으로 등 단어의 종류가 너무 커져버리므로 
# 단어는 고빈도 단위로 항상 자르는데 OUT OF VOCA 처리되는 어절이 너무많음 따라서 형태소 단위로 해야한다.
# NLTK 한국어 패키지 형태소분석 패키지 사용해서 => 형태소 임베딩 나온거 -> 그이후 ㄷ동일하게 영어처럼 진행

# Your code here


model = keras.Sequential([
    keras.layers.Embedding(vocab_size,50),
    keras.layers.Conv1D(32, 3,padding = "same",activation=tf.nn.relu),## 3개단어를 보면서 피쳐맵을 만든다.
    keras.layers.GlobalMaxPool1D(),
    keras.layers.Dense(2, activation= tf.nn.softmax)
   
    ## 7*7*64를 쫙펴서 덴스레이어 
])





아래는 학습 결과를 시각화해주고, 성능을 측정하는 함수들입니다.

In [0]:
def plot_loss(history):
  plt.figure(figsize=(6,5))
  val = plt.plot(history.epoch, history.history['val_loss'],
                 '--', label='Test')
  plt.plot(history.epoch, history.history['loss'], color=val[0].get_color(),
           label='Train')

  plt.xlabel('Epochs')
  plt.ylabel("Loss")
  plt.legend()

  plt.xlim([0,max(history.epoch)])
  
def eval_model(model):
  test_loss, test_acc = model.evaluate(test_inputs, test_labels)
  print('Test accuracy:', test_acc)

만들어진 모델을 학습시켜보겠습니다.

In [50]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_inputs,
          train_labels,
          epochs=10,
          validation_data=(test_inputs, test_labels)
         )

plot_loss(history)
eval_model(model)

NameError: ignored

## Pretrained word vectors

이번에는 만들어진 모델에 미리 학습된 단어 벡터를 적용해보겠습니다.

단어 벡터는 GloVe 벡터를 사용할 것입니다.

벡터 파일을 다운로드 받고 압축을 풀어보겠습니다.

파일이 어떻게 구성되어 있는지 볼까요?

glove 단어, 벡터 있는애 워드투백과 비슷

전체 단어에 대한 임베딩
글로브있는단어, 우리가 가지고 있는단어 비교
글로브벡터가지고 우리 모델을 학습시킨다.

In [51]:
if not os.path.exists('glove.6B.zip'):
    ! wget http://nlp.stanford.edu/data/glove.6B.zip
if not os.path.exists('glove.6B.50d.txt'):
    ! unzip glove.6B.zip
    
! head glove.6B.50d.txt ## 6B 얼마만큼 큰 말뭉치로 학습 시켰다. 50d 이벡터 차원이 50차원

--2019-08-01 08:12:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-08-01 08:12:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-08-01 08:12:40--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

[0]이 단어고 나머지는 벡터값

GloVe 벡터를 불러와서 임베딩 행렬을 초기화해보겠습니다.



1.   GloVe 파일을 읽고, 각 줄에서 단어(1번째 토큰)와 벡터를 이루는 숫자들(2번째 이후 토큰들)을 분리
2.   벡터를 이루는 숫자들을 numpy 행렬로 변환 (numpy의 asarray 함수 활용)
3.   단어와 벡터를 연결하는 dictionary 자료구조 구성 (단어 -> 벡터)
4.   모든 단어들에 대한 임베딩 행렬을 무작위로 생성 (vocab_size X 50 크기의 numpy 행렬)
5.   임베딩 행렬에서, GloVe 벡터가 존재하는 단어들만 해당 GloVe 벡터로 대체



In [0]:
def load_glove_embeddings(path):   ##키 : 단어 // value : 인베딩벡터 이걸 로딩 
     embeddings = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f: ## 텍스트 파일 한줄씩 라인으로
            # Your code here
            values = linke.strip().split() ## strip 줄바꿈 제거, # split() 띄어쓰기 단위
            w = values[0] ## 첫번째깞이 워드
            vectors = np.asarray(values[1:], dypte="float32") ## 넘파이 벡터로 만들고
            embeddings[w] = vectors
                                                                        
                                                                        ## 정규분포가 아닌 균등분포
    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size, 50)) ## 무작위로 초기화 전체 단어에 대한 임베딩 행렬 초기화 5000단어, 글로브벡터 50차원에 맟춰서 # 임베딩행렬 제일 첫단 분산 영향 x
    num_loaded = 0  #몇개나 불러왔는지 체크
    
    for w, i in word_index.items(): ## 단어가 있으면 인베딩벡터값으로 바꾼다.
        # Your code here
        # emvedding[] 이런식하면 키없으면 오류나서
        v = embddings.get(w)
        if 
        num_loaded = num_loaded + 1
            
    print('Successfully loaded pretrained embeddings for '
          f'{num_loaded}/{vocab_size} words.')
    embedding_matrix = embedding_matrix.astype(np.float32)
    return embedding_matrix

embedding_matrix = load_glove_embeddings('glove.6B.50d.txt')

앞서 사용된 모델에서, Embedding layer의 값을 위에서 생성한 임베딩 행렬로 초기화해봅시다.

(keras.initializers.Constant 클래스 활용)


In [0]:
# Your code here

# 우리가 만든 임베딩 함수로 초기화 한다 => glove_vec

# 성능은 구데기 글로브벡터랑 우리가 사용하는거랑 도메인이 안맞는다.

# 지금까지는 지도학습방식
# 정답사람시켜서찾은건 2000문장

# 데이터 구하기 개빢셈

# 어떻게 하면 비지도랑 결합해서 지도학습 성능 끌어올릴까?

# 다른 모델에서 학습을 시키고 새로운 모델에 다시 학습을 시킴 (트랜스펄러닝)

# 비지도 학습으로 1차학습 => 실제 원하는데이터로 추가 학습 => 데이터가 적더라도 높은 성능 보인다.

# 글로브 벡터( 60억 토큰 말뭉치 학습) 존나 큰거

# 우리가 원하는 모델에 글로브벡터 가져옴 두개 모델간 학습 목적이 맞지않아서 성능이 구데기 

# 트랜스퍼 러닝할떄는 첫 태스크 두번쨰 태스크 얼마나 유사성이 있느냐

# 텐서플로 허브란

# 데이터 많이 확보할수 업스니까 기존 모델 재활용 => 새로운 모델에 활용 이게 중요한 이슈

# 예전에는 깃허브 다운이 곧 모델만

# 이제는 모델이랑 모델 학습시키는 웨이트 같이 배포

# 텐서플로서 유지보수 => 전이학습할수 있게 해논 플랫폼 => 텐서플로 허브

# 텍스트 임베딩( 텍스트 -> 벡터)

# 학습된 모델, 코드 쭉정리 universal-sentence-encoder-large문장을 잘표현할수 있는목적
# 문장, 문단 텍스트를 벡터로 만들수 있따. 

# cnn모델목적 텍스트 들어왔을떄 벡터를 얻는거

# 텐서플로 받은 객체 안에 맨오른쪽 2칸짜리 전까지 그래프 다들어있다.

# 문장넣으면 벡터 나옴 덴서 모델 만들어서 분류해서 학습시키면 된다.  ## 덴서말고 svm

# 버트 = 순전히 비지도로 학습한거 단어가 주어졌을떄 다음단어 예측 (빈칸채우기) 학습시킬떈

# 문장을 주고 모델에 빈칸 단어 뭘까를 학습 비지도학습

# 문장간의 상관관계 비지도학습 => 버트 모델 줜나 큼 + DENSE레이어 하면 댐

# NER개체마다 벡터가나와야되는데 그것도 버트로 대체

# 버트 줜나어려움 프로젝트 사용에 도움되랑 이거야

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_inputs,
          train_labels,
          epochs=10,
          validation_data=(test_inputs, test_labels)
         )

plot_loss(history)
eval_model(model)